# Laboratorio 1 - Analisis de ventas Big Data - Ivan Gonzalo Tapia



### Consignas:
Preguntas a responder en el análisis
—————————————————————————————————————————
1. Comportamiento de las ventas en los distintos meses:

  ● ¿Cómo variaron las ventas a lo largo de los diferentes meses?

  ● ¿Hubo algún mes que se destacó en términos de ventas?

  ● ¿Cuál es el ingreso total generado por mes?
2. Optimización de la publicidad y patrón de ventas por hora:

  ● ¿Cuál es el momento ideal del día para presentar la publicidad y aumentar la
  probabilidad de compra?

  ● ¿Cómo cambian los patrones de ventas por hora a lo largo del año?

  ● ¿Hay modificaciones en los patrones de ventas durante las horas de mayor actividad
  en los distintos meses?

3. Distribución de ventas por ubicación:

  ● ¿En qué ciudades se han registrado las mayores ventas?

  ● ¿Cómo se comparan las ventas en diferentes estados o regiones?

  ● ¿Existe variación en las ventas por estado a lo largo de los meses?

4. Análisis del producto más vendido:

  ● ¿Cuál es el producto más vendido en general y en cada mes?

  ● ¿Qué factores crees que han contribuido al éxito de ese producto en particular?

5. Tendencia ventas:

  ● ¿Existe alguna tendencia o patrón en las ventas que se repita a lo largo de los días
  en los diferentes meses?

  ● ¿Cómo varían las ventas a lo largo de los diferentes días de la semana?

  ● ¿Se observa alguna diferencia significativa en las ventas entre los días laborables y
  los fines de semana?

6. Impacto de eventos especiales en las ventas:

  ● ¿Se ha observado algún aumento o disminución significativa en las ventas en días
  cercanos a eventos especiales, como días festivos?

  ● ¿Qué eventos específicos han tenido un impacto notable en el comportamiento de
  las ventas y cómo se manifestó ese impacto?

Pregunta Adicional
—————————————————————————————————————————

   - Además de las preguntas mencionadas anteriormente, te invito a pensar en una pregunta
  adicional que podría ser relevante para analizar los datos de ventas mensuales. Esta
  pregunta debe estar basada en la información que se proporciona en los conjuntos de
  datos. Puede estar relacionada con tendencias, comparaciones, patrones o cualquier otro
  aspecto que despierte tu interés. Trata de ser creativo/a y pensar en cómo podrías explorar
  aún más estos datos!


Pasos del Análisis
—————————————————————————————————————————
1. Carga de datos:
  - Cargar los datos de ventas de cada archivo CSV.
2. Limpieza de los datos:
  - Eliminar datos no numéricos.
  - Eliminar filas incompletas
3. Preparación de los datos:
  - Ajustar los tipos de datos de cada atributo
  - Extraer características importantes para el análisis como meses, horas, ciudades etc.
4. Análisis de datos:
  Ejemplo:
  - Examinar las horas del día en que las ventas son más frecuentes.
  - Determinar el mejor momento para mostrar publicidad.
5. Visualización de datos:
  - Utilizar gráficos y visualizaciones para representar los resultados de los análisis.
  - Crear gráficos de barras, líneas, mapas, tablas, etc, según corresponda.

## Import de librerias y recursos a utilizar

In [ ]:
# Librerias necesarias para el desarrollo del proyecto
import os
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px


In [ ]:
# Directorio propio para importar los datasets necesarios

path = '/content/drive/MyDrive/UPSO ML & BD/Dataset de ventas2'

archivos = os.listdir(path)
archivos

['Dataset_de_ventas_Abril.csv',
 'Dataset_de_ventas_Agosto.csv',
 'Dataset_de_ventas_Diciembre.csv',
 'Dataset_de_ventas_Enero.csv',
 'Dataset_de_ventas_Febrero.csv',
 'Dataset_de_ventas_Julio.csv',
 'Dataset_de_ventas_Junio.csv',
 'Dataset_de_ventas_Marzo.csv',
 'Dataset_de_ventas_Mayo.csv',
 'Dataset_de_ventas_Noviembre.csv',
 'Dataset_de_ventas_Octubre.csv',
 'Dataset_de_ventas_Septiembre.csv']

## Preparacion y limpieza de lo datos

In [ ]:
# Ahora voy a crear un diccionario para guardar estos archivos(datasets), para luego hacer un df

dataframes = {}

for archivo in archivos:
  if archivo.endswith('csv'):
    mes = archivo.split('_')[3].split('.')[0] # obtener el nombre del mes solo
    #aca obtenemos la ruta completa de donde se encuentra el csv de cada mes
    path_completo = os.path.join(path, archivo) # obtener ruta de acceso completa
    df = pd.read_csv(path_completo)   #cargar el archivo csv en un df
    #vamos a trabajar sobre una copia del df original
    df_ventas = df.copy()


    # SANITIZAR CADA DF ANTES DE CARGARLO
    df_ventas['Cantidad Pedida'] = pd.to_numeric(df_ventas['Cantidad Pedida'], errors = 'coerce') #errors = coerce transfora lo
    df_ventas['Precio Unitario'] = pd.to_numeric(df_ventas['Precio Unitario'], errors = 'coerce')
    #esta es una mascara que me genera un true si hay un nan, y sino hay nan me pone un false
    df_ventas = df_ventas.loc[~df_ventas['Cantidad Pedida'].isna()]
    df_ventas = df_ventas.loc[~df_ventas['Precio Unitario'].isna()]
    #aca elimino los nan's
    df_ventas = df_ventas.dropna()
    #aca voy a pasar todos los datos al tipo q corresponda y agregar las columnas que sean necesarias
    df_ventas['Cantidad Pedida'] = df_ventas['Cantidad Pedida'].astype(int) #esta funcion me transforma todos los valores a numericos a int
    df_ventas['Precio Unitario'] = df_ventas['Precio Unitario'].astype(float) #esta funcion me transforma todos los valores a numericos a float

    # FINALIZA SANITIZACION

    # CREO COLUMNAS NECESARIAS PARA ANALISIS
    df_ventas['Fecha de Pedido'] = pd.to_datetime(df_ventas['Fecha de Pedido'], format='%m/%d/%y %H:%M')

    # Obtener la hora y minutos como número entero de 4 dígitos (por ejemplo, 1430)
    df_ventas['Hora de Pedido'] = df_ventas['Fecha de Pedido'].dt.strftime('%H%M').astype(int)
    df_ventas['Estado'] = df_ventas['Dirección de Envio'].str.split(', ').str[-1].str.split(' ').str[0]
    df_ventas['Ciudad'] = df_ventas['Dirección de Envio'].str.split(', ').str[1]
    df_ventas['Dia de Pedido'] = df_ventas['Fecha de Pedido'].dt.strftime('%m-%d')
    #FINALIZA CREACION DE COLUMNAS

    #aca sucede la magia, vamos agregando cada mes al df ventas
    #a cada mes le cargo su dataset correspondiente, por eso uso la variable mes
    dataframes[mes] = df_ventas
    #convierto



dataframe_e = dataframes['Enero'].sort_values(by='Dia de Pedido')
dataframe_e.columns


Index(['ID de Pedido', 'Producto', 'Cantidad Pedida', 'Precio Unitario',
       'Fecha de Pedido', 'Dirección de Envio', 'Hora de Pedido', 'Estado',
       'Ciudad', 'Dia de Pedido'],
      dtype='object')

In [ ]:
dataframes['Febrero']

,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Fecha de Pedido,Dirección de Envio
0,150502,iPhone,1,700.00,2019-02-18 01:35:00,"866 Spruce St, Portland, ME 04101"
1,150503,AA Batteries (4-pack),1,3.84,2019-02-13 07:24:00,"18 13th St, San Francisco, CA 94016"
2,150504,27in 4K Gaming Monitor,1,389.99,2019-02-18 09:46:00,"52 6th St, New York City, NY 10001"
3,150505,Lightning Charging Cable,1,14.95,2019-02-02 16:47:00,"129 Cherry St, Atlanta, GA 30301"
4,150506,AA Batteries (4-pack),2,3.84,2019-02-28 20:32:00,"548 Lincoln St, Seattle, WA 98101"
...,...,...,...,...,...,...
12031,162004,Apple Airpods Headphones,1,150.00,2019-02-12 22:02:00,"227 Church St, San Francisco, CA 94016"
12032,162005,AAA Batteries (4-pack),2,2.99,2019-02-04 20:44:00,"417 Jefferson St, Los Angeles, CA 90001"
12033,162006,USB-C Charging Cable,1,11.95,2019-02-24 06:31:00,"498 8th St, Atlanta, GA 30301"
12034,162007,USB-C Charging Cable,1,11.95,2019-02-24 19:09:00,"715 7th St, Dallas, TX 75001"


In [ ]:
# Ordenar los df por meses

# meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']

# for mes in meses:
#   dataframe_mes = dataframes[mes]

## Analisis y graficos de datos

In [ ]:
# Funcion para obtener los datos de dia y hora de los pedidos

def extraer_datos(datos):
  datos['Fecha de Pedido'] = pd.to_datetime(datos['Fecha de Pedido'])
  datos['Hora'] = df_ventas['Fecha de Pedido'].dt.hour
  datos['Dia'] = df_ventas['Fecha de Pedido'].dt.day
  return datos

In [ ]:
# Ordenar los meses
meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']

totales_ventas = []


for mes in meses:
  dataframe_mes = dataframes[mes]
  dataframes_mes = extraer_datos(dataframe_mes)

  #aca sumo el total de ventas de cada mes
  total_ventas_mes = dataframe_mes['Cantidad Pedida'].sum() #aca obtengo el total de ventas de cada df(x mes)
  totales_ventas.append(total_ventas_mes)                   #append al array de la suma de total de ventas x cada mes(creo un array con las ventas de cada mes)


,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Fecha de Pedido,Dirección de Envio,Hora de Pedido,Estado,Ciudad,Dia de Pedido,Hora,Dia
0,295665,Macbook Pro Laptop,1,1700.00,2019-12-30 00:01:00,"136 Church St, New York City, NY 10001",1,NY,New York City,12-30,14.0,17.0
1,295666,LG Washing Machine,1,600.00,2019-12-29 07:03:00,"562 2nd St, New York City, NY 10001",703,NY,New York City,12-29,10.0,29.0
2,295667,USB-C Charging Cable,1,11.95,2019-12-12 18:21:00,"277 Main St, New York City, NY 10001",1821,NY,New York City,12-12,17.0,16.0
3,295668,27in FHD Monitor,1,149.99,2019-12-22 15:13:00,"410 6th St, San Francisco, CA 94016",1513,CA,San Francisco,12-22,7.0,27.0
4,295669,USB-C Charging Cable,1,11.95,2019-12-18 12:38:00,"43 Hill St, Atlanta, GA 30301",1238,GA,Atlanta,12-18,19.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
25112,319666,Lightning Charging Cable,1,14.95,2019-12-11 20:58:00,"14 Madison St, San Francisco, CA 94016",2058,CA,San Francisco,12-11,NaN,NaN
25113,319667,AA Batteries (4-pack),2,3.84,2019-12-01 12:01:00,"549 Willow St, Los Angeles, CA 90001",1201,CA,Los Angeles,12-01,NaN,NaN
25114,319668,Vareebadd Phone,1,400.00,2019-12-09 06:43:00,"273 Wilson St, Seattle, WA 98101",643,WA,Seattle,12-09,NaN,NaN
25115,319669,Wired Headphones,1,11.99,2019-12-03 10:39:00,"778 River St, Dallas, TX 75001",1039,TX,Dallas,12-03,NaN,NaN


### 1. Comportamiento de las ventas en los distintos meses:

  ● ¿Cómo variaron las ventas a lo largo de los diferentes meses?

  ● ¿Hubo algún mes que se destacó en términos de ventas?

  ● ¿Cuál es el ingreso total generado por mes?

In [ ]:
# Creo la figura con plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Bar(
    x=meses,
    y=totales_ventas,
    yaxis="y",
    name="Unidades vendidas",
    marker =dict(color=totales_ventas,colorscale= 'RdYlGn', colorbar= dict(title= "Total de ventas")),
    text = totales_ventas,
    textposition = 'auto'

))

fig.update_layout(
    title="Unidades vendidas por mes",
    xaxis=dict(title="Meses"),
    yaxis=dict(title="Unidades vendidas", side = "left")
)

fig.show()

In [ ]:
import plotly.graph_objects as go

sizes = []
colors = []

#Hago un for para recorrer la cantidad total de ventas, y voy a dividir por una constante(1000), para hacer las burbujas de color acorde a la cantidad de unidades vendidas por mes
for total in totales_ventas:
  s = 35
  c = total / 1000
  sizes.append(s)
  colors.append(c)

fig = go.Figure(data=[go.Scatter(
    x=meses,
    y=totales_ventas,
    mode='markers',
    marker=dict(
        color=colors,
        size= sizes,
        showscale=True,
        colorbar= dict(title= "Ventas en miles")
        )
)])

fig.update_layout(

  title="Cantidad de productos vendidos por mes",
  xaxis=dict(title="Meses", tickvals=meses),  # Para que en el eje x se muestren todos los valores uso tickvals=Horas
  yaxis=dict(title="Unidades vendidas", side="left"),
  legend=dict(y=0.3, yanchor='bottom')
)


fig.show()

In [ ]:
ingresos_totales = []

for mes in meses:
  total_ventas_mes = (dataframes[mes]['Cantidad Pedida']*dataframes[mes]['Precio Unitario']).sum().round(2)
  ingresos_totales.append(total_ventas_mes)

# Crear el objeto figura
fig = go.Figure()

#Configuro titulos y ejes del grafico
fig.add_trace(go.Bar(
        x = meses,
        y = ingresos_totales,
        yaxis= 'y',
        name = 'Ingreso Total',
        marker = dict(color= ingresos_totales, colorscale='balance', colorbar=dict(title = 'Total de ventas')),
        text = ingresos_totales,
        textposition = 'outside',
)
)

fig.update_layout(
    title = 'Ingresos totales por mes',
    xaxis = dict(title='Meses'),
    yaxis = dict(title='Ingreso total',side='left')
)

#Muestro el grafico
fig.show()

### 2. Optimización de la publicidad y patrón de ventas por hora:

  ● ¿Cuál es el momento ideal del día para presentar la publicidad y aumentar la
  probabilidad de compra?

  ● ¿Cómo cambian los patrones de ventas por hora a lo largo del año?

  ● ¿Hay modificaciones en los patrones de ventas durante las horas de mayor actividad
  en los distintos meses?

In [ ]:
# Creo el objeto figura
Horas = list(range(24))
fig = go.Figure()


for mes in meses:

  Ventas_por_hora= dataframes[mes].groupby('Hora')['Cantidad Pedida'].sum()

  # Agrego al objeto figura un grafico de de linea con las horas y las ventas por hora
  fig.add_trace(go.Scatter(

      x=Horas,
      y=Ventas_por_hora,
      mode='lines',
      name=mes,
      line=dict(shape='spline', smoothing=1.3, width=2)))


  # Configuro los ejes y los títulos
fig.update_layout(

    title="Unidades vendidas por hora",
    xaxis=dict(title="Hora", tickvals=Horas),  # Para que en el eje x se muestren todos los valores uso tickvals=Horas
    yaxis=dict(title="Unidades vendidas", side="left"),
    legend=dict(y=0.3, yanchor='bottom')),



fig.show()

In [ ]:
fig = go.Figure()

horas_analizadas = [12,19,14]

for hora in horas_analizadas:
  total_productos_vendidos = []

  for mes in meses:
    dataframe_mes = dataframes[mes]
    dataframes_mes = extraer_datos(dataframe_mes)

    ventas_hora = dataframe_mes[dataframe_mes['Hora'] == hora]

    total_productos_vendidos.append(ventas_hora['Cantidad Pedida'].sum())

  fig.add_trace(go.Scatter(

      x=meses,
      y=total_productos_vendidos,
      mode='lines',
      name=f'{hora} horas',
      line = dict(width= 2, shape= 'spline')
  ))


# Configuro los ejes y los títulos
fig.update_layout(

  title="Unidades vendidas por hora",
  xaxis=dict(title="Hora", tickvals=Horas),  # Para que en el eje x se muestren todos los valores uso tickvals=Horas
  yaxis=dict(title="Unidades vendidas", side="left"),
  legend=dict(y=0.3, yanchor='bottom')
)


fig.show()

In [ ]:


fig = go.Figure()
fig.add_trace(go.Bar(
    x=meses,
    y=total_productos_vendidos,
    name='Total de unidades vendidas',
    marker_color='indianred'
))
fig.add_trace(go.Bar(
    x=meses,
    y=Ventas_por_hora,
    name='Ventas por hora',
    marker_color='lightsalmon'
))

fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()

In [ ]:
# import plotly.graph_objects as go

# totales_ventas = []

# for mes in meses:

#   total_ventas_mes = dataframes[mes]['Cantidad Pedida'].sum()
#   totales_ventas.append(total_ventas_mes)

# # Crear el objeto figura
# fig = go.Figure()

# #Configuro titulos y ejes del grafico
# fig.add_trace(go.Bar(
#         x = meses,
#         y = totales_ventas,
#         yaxis= 'y',
#         name = 'Unidades Vendidas',
#         marker = dict(color= totales_ventas, colorscale='RdYlGn', colorbar=dict(title = 'Total de ventas')),
#         text = totales_ventas,
#         textposition = 'auto',
# )
# )

# fig.update_layout(
#     title = 'Unidades Vendidas por mes',
#     xaxis = dict(title='Meses'),
#     yaxis = dict(title='Unidades Vendidas',side='left')
# )

# #Muestro el grafico
# fig.show()

### 2 - Optimizacion de las ventas por hora


In [ ]:
ventas_hora = []
# Agrupar las ventas por hora y calcular el total de ventas por hora
for mes in meses:
  ventas_por_hora = dataframes[mes].groupby(dataframes[mes]['Hora de Pedido'])['Cantidad Pedida'].sum().reset_index()
  ventas_hora.append(ventas_por_hora)
# Encontrar la hora con las ventas más altas
hora_max_ventas = ventas_por_hora.loc[ventas_por_hora['Cantidad Pedida'].idxmax()]['Hora de Pedido']
print(f"La hora ideal para presentar la publicidad es a las {hora_max_ventas} horas.")

La hora ideal para presentar la publicidad es a las 1145 horas.


In [ ]:
# Crear un gráfico de barras apiladas para visualizar los patrones de ventas por hora a lo largo del año
import plotly.express as px

# Agrupar las ventas por ciudad y calcular el total de ventas por ciudad
ventas_por_ciudad = df.groupby('Ciudad')['Cantidad Pedida'].sum().reset_index()

# Crear un gráfico de barras para visualizar las ventas por ciudad
fig = px.bar(ventas_por_ciudad, x='Ciudad', y='Cantidad Pedida', labels={'Cantidad Pedida': 'Ventas'})
fig.update_layout(title="Ventas por ciudad")
fig.show()

### 3. Distribución de ventas por ubicación:

  ● ¿En qué ciudades se han registrado las mayores ventas?

  ● ¿Cómo se comparan las ventas en diferentes estados o regiones?

  ● ¿Existe variación en las ventas por estado a lo largo de los meses?


In [ ]:
# Voy a investigar las ventas por ciudad, obteniendo los datos de la columna ciudad que cree en el procesamiento de datos
for mes in meses:
  ventas_por_ciudad =  dataframes[mes].groupby('Ciudad')['Cantidad Pedida'].sum().reset_index(inplace=False)

ventas_por_ciudad

,Ciudad,Cantidad Pedida
0,Atlanta,2224
1,Austin,1520
2,Boston,3029
3,Dallas,2300
4,Los Angeles,4373
5,New York City,3828
6,Portland,1859
7,San Francisco,6786
8,Seattle,2202


In [ ]:
import plotly.graph_objs as go

# Calcular los máximos por ciudad
maximos_por_ciudad = ventas_por_ciudad.groupby('Ciudad')['Cantidad Pedida'].max().reset_index()

# Crear el gráfico
fig = go.Figure()

ciudades = ventas_por_ciudad['Ciudad']
cantidades = ventas_por_ciudad['Cantidad Pedida']
colores_ciudad = px.colors.qualitative.Set1
# Agregar el gráfico de líneas
fig.add_trace(go.Scatter(
    x=ciudades,
    y=cantidades,
    mode='lines',
    name="Variacion de unidad vendidas",
    line=dict(width=2, shape='spline')
))

# Marcar los puntos máximos por ciudad
for _, row in maximos_por_ciudad.iterrows():
    ciudad_max = row['Ciudad']
    cantidad_max = row['Cantidad Pedida']
    color_max = color_por_ciudad[ciudad_max]  # Obtener el color correspondiente a la ciudad máxima
    fig.add_trace(go.Scatter(
        x=[ciudad_max],
        y=[cantidad_max],
        mode='markers',
        name=f'Máximo ({ciudad_max})',
        marker=dict(size=10, color=color_max)  # Asignar el color al gráfico de dispersión
    ))


# Configurar ejes y título
fig.update_layout(
    title="Unidades vendidas por ciudad",
    xaxis=dict(title="Ciudad"),
    yaxis=dict(title="Unidades vendidas"),
)
fig.show()
# Crear el gráfico de pastel interactivo
fig_pie = px.pie(df_pie, names='Ciudad', values='Ventas', color='Ciudad', color_discrete_map=color_por_ciudad,
             labels={'Ciudad': 'Ciudad', 'Ventas': 'Ventas'},
             hover_data=['Proporción'],
             title='Distribución de Ventas por Ciudad')

# Agregar etiquetas de porcentaje
fig_pie.update_traces(textinfo='percent+label')

# Destacar la ciudad con más ventas
fig_pie.update_traces(pull=[0.1] + [0] * (len(ciudades) - 1))

# Configurar el diseño
fig_pie.update_layout(legend_title_text='Ciudades')

# Mostrar el gráfico

fig_pie.show()

fig_bar = go.Figure()
for ciudad in ciudades:
    color = color_por_ciudad[ciudad]  # Obtener el color correspondiente a la ciudad
    fig_bar.add_trace(go.Bar(
        x=[ciudad],
        y=[cantidades[ciudades == ciudad].values[0]],  # Obtener la cantidad para la ciudad específica
        name=ciudad,
        marker_color=color  # Asignar el color al gráfico de barras
    ))

# Configurar ejes y título
fig_bar.update_layout(
    title="Unidades vendidas por ciudad",
    xaxis=dict(title="Ciudad"),
    yaxis=dict(title="Unidades vendidas"),
)

fig_bar.show()

In [ ]:
import plotly.graph_objs as go

# Calcular los máximos por ciudad
maximos_por_ciudad = ventas_por_ciudad.groupby('Ciudad')['Cantidad Pedida'].max().reset_index()

# Crear el gráfico
fig = go.Figure()

ciudades = ventas_por_ciudad['Ciudad']
cantidades = ventas_por_ciudad['Cantidad Pedida']
colores_ciudad = px.colors.qualitative.Set1

# Agregar el gráfico de líneas
fig.add_trace(go.Scatter(
    x=ciudades,
    y=cantidades,
    mode='lines',
    name="Variacion de unidad vendidas",
    line=dict(width=2, shape='spline')
))

# Marcar los puntos máximos por ciudad
for _, row in maximos_por_ciudad.iterrows():
    ciudad_max = row['Ciudad']
    cantidad_max = row['Cantidad Pedida']
    color_max = color_por_ciudad[ciudad_max]  # Obtener el color correspondiente a la ciudad máxima
    fig.add_trace(go.Scatter(
        x=[ciudad_max],
        y=[cantidad_max],
        mode='markers',
        name=f'Máximo ({ciudad_max})',
        marker=dict(size=10, color=color_max)  # Asignar el color al gráfico de dispersión
    ))

# Configurar ejes y título
fig.update_layout(
    title="Unidades vendidas por ciudad",
    xaxis=dict(title="Ciudad"),
    yaxis=dict(title="Unidades vendidas"),
)
fig.show()






In [ ]:
import plotly.graph_objs as go

# Crear el gráfico
fig = go.Figure()

ciudades = ventas_por_ciudad['Ciudad']
cantidades = ventas_por_ciudad['Cantidad Pedida']

# Obtener una paleta de colores similar a la usada en Plotly Express
colores_ciudad = px.colors.qualitative.Set1

# Crear un diccionario que mapee cada ciudad a su color correspondiente
color_por_ciudad = {}
for i, ciudad in enumerate(ciudades):
    color = colores_ciudad[i % len(colores_ciudad)]  # Ciclar colores de la paleta
    color_por_ciudad[ciudad] = color

# Agregar el gráfico de barras con colores por ciudad
for ciudad in ciudades:
    color = color_por_ciudad[ciudad]  # Obtener el color correspondiente a la ciudad
    fig.add_trace(go.Bar(
        x=[ciudad],
        y=[cantidades[ciudades == ciudad].values[0]],  # Obtener la cantidad para la ciudad específica
        name=ciudad,
        marker_color=color  # Asignar el color al gráfico de barras
    ))

# Agregar el gráfico de dispersión con los máximos por ciudad
for _, row in maximos_por_ciudad.iterrows():
    ciudad_max = row['Ciudad']
    cantidad_max = row['Cantidad Pedida']
    color = "red"

# Configurar ejes y título
fig.update_layout(
    title="Unidades vendidas por ciudad",
    xaxis=dict(title="Ciudad"),
    yaxis=dict(title="Unidades vendidas"),
)

fig.show()


In [ ]:
import plotly.express as px

# Calcular las proporciones de ventas por ciudad
ventas_por_ciudad = ventas_por_ciudad.sort_values(by='Cantidad Pedida', ascending=False)
ciudades = ventas_por_ciudad['Ciudad']
cantidades = ventas_por_ciudad['Cantidad Pedida']
total_ventas = cantidades.sum()
proporciones = cantidades / total_ventas

# Crear un DataFrame para el gráfico
df_pie = pd.DataFrame({'Ciudad': ciudades, 'Ventas': cantidades, 'Proporción': proporciones})

# Obtener una paleta de colores similar a la usada en Plotly Express
colores_ciudad = px.colors.qualitative.Set1

# Crear un diccionario que mapee cada ciudad a su color correspondiente
color_por_ciudad = {}
for i, ciudad in enumerate(ciudades):
    color = colores_ciudad[i % len(colores_ciudad)]  # Ciclar colores de la paleta
    color_por_ciudad[ciudad] = color

# Crear el gráfico de pastel interactivo
fig = px.pie(df_pie, names='Ciudad', values='Ventas', color='Ciudad', color_discrete_map=color_por_ciudad,
             labels={'Ciudad': 'Ciudad', 'Ventas': 'Ventas'},
             hover_data=['Proporción'],
             title='Distribución de Ventas por Ciudad')

# Agregar etiquetas de porcentaje
fig.update_traces(textinfo='percent+label')

# Destacar la ciudad con más ventas
fig.update_traces(pull=[0.1] + [0] * (len(ciudades) - 1))

# Configurar el diseño
fig.update_layout(legend_title_text='Ciudades')

# Mostrar el gráfico
fig.show()




In [ ]:
# Ahora voy a investigar las ventas totales por estado
for mes in meses:
  ventas_por_estado = dataframes[mes].groupby('Estado')['Cantidad Pedida'].sum()
  ventas_por_estado = ventas_por_estado.reset_index(inplace=False)

ventas_por_estado

,Estado,Cantidad Pedida
0,CA,11159
1,GA,2224
2,MA,3029
3,ME,352
4,NY,3828
5,OR,1507
6,TX,3820
7,WA,2202


In [ ]:
import plotly.graph_objs as go

# Calcular los máximos por estado
maximos_por_estado = ventas_por_estado.groupby('Estado')['Cantidad Pedida'].max().reset_index()
maximos_por_estado = maximos_por_estado.sort_values(by='Cantidad Pedida', ascending=True)
# Crear el gráfico
fig = go.Figure()

estados = ventas_por_estado['Estado']
cantidad = ventas_por_estado['Cantidad Pedida']

# Agregar el gráfico de líneas
fig.add_trace(go.Scatter(
    x=estados,
    y=cantidad,
    mode='lines',
    name="Ciudad",
    line=dict(width=2, shape='spline')
))

# Marcar los puntos máximos por estado
for _, row in maximos_por_estado.iterrows():
    estado_max = row['Estado']
    cantidad_max = row['Cantidad Pedida']
    fig.add_trace(go.Scatter(
        x=[estado_max],
        y=[cantidad_max],
        mode='markers',
        name=f'Máximo ({estado_max})',
        marker=dict(size=10)
    ))

# Configurar ejes y título
fig.update_layout(
    title="Unidades vendidas por estado",
    xaxis=dict(title="Ciudad"),
    yaxis=dict(title="Unidades vendidas"),
)

fig.show()

In [ ]:
import plotly.express as px


# Hago un plot de Estados Unidos
fig = px.choropleth(ventas_por_estado, locations='Estado', locationmode='USA-states',color='Cantidad Pedida',
                           color_continuous_scale="RdYlGn",
                           scope="usa",
                          title= 'Ventas por estado'
                          )

fig.show()

In [ ]:
ventas_por_estado_mes = {}
estados = []


fig = go.Figure()

for estado in dataframes['Enero']['Estado']:
  if estado not in estados:
    estados.append(estado)

for mes in meses:
  ventas_por_estado_mes[mes] = dataframes[mes].groupby('Estado')['Cantidad Pedida'].sum()
  ventas_por_estado_mes[mes] = ventas_por_estado_mes[mes].reset_index(inplace=False)

# Agrego al objeto figura un gráfico de línea con las horas y las ventas por hora
  fig.add_trace(go.Bar(
      x=ventas_por_estado_mes[mes]['Estado'],
      y=ventas_por_estado_mes[mes]['Cantidad Pedida'],
      name= mes,
  ));

# Configuro ejes y los títulos
fig.update_layout(
    title="Productos vendidos por mes en cada estado",
    xaxis=dict(title="Estados"),
    yaxis=dict(title="Ventas", side="left"),
);

fig.show()

In [ ]:
import pandas as pd

# Supongamos que tienes un diccionario llamado "dataframes" con los DataFrames correspondientes a cada mes
# Puedes cargar los datos desde tu fuente, por ejemplo:
# dataframes = {'Enero': df_enero, 'Febrero': df_febrero, ...}

# Define una función para extraer el estado a partir de la columna "Dirección de Envío"
def obtener_estado(direccion):
    partes = direccion.split(", ")
    if len(partes) > 1:
        return partes[-1]
    else:
        return None

# Inicializa un DataFrame vacío para almacenar los resultados
df_estado_total = pd.DataFrame(columns=['Estado', 'Cantidad Pedida Total'])

# Itera a través de los DataFrames de cada mes en el diccionario
for mes, df_mes in dataframes.items():
    # Aplica la función para obtener el estado y crea una nueva columna "Estado"
    df_mes['Estado'] = df_mes['Dirección de Envio'].apply(obtener_estado)

    # Agrupa por "Estado" y suma la "Cantidad Pedida" para obtener la cantidad total por estado en este mes
    df_mes_estado_total = df_mes.groupby('Estado')['Cantidad Pedida'].sum().reset_index()

    # Concatena los resultados con el DataFrame total
    df_estado_total = pd.concat([df_estado_total, df_mes_estado_total], ignore_index=True)

# Agrupa el DataFrame total por "Estado" y suma la "Cantidad Pedida" para obtener la cantidad total por estado en todos los meses
df_estado_total = df_estado_total.groupby('Estado')['Cantidad Pedida'].sum().reset_index()

# df_estado_total ahora contiene la cantidad pedida total por cada estado para todos los meses
df_estado_total.head()






,Estado,Cantidad Pedida
0,CA 90001,33289.0
1,CA 94016,50239.0
2,GA 30301,16602.0
3,MA 02215,22528.0
4,ME 04101,2750.0


### 4. Análisis del producto más vendido:
  ● ¿Cuál es el producto más vendido en general y en cada mes?

  ● ¿Qué factores crees que han contribuido al éxito de ese producto en particular?

### 5. Tendencia ventas:

  ● ¿Existe alguna tendencia o patrón en las ventas que se repita a lo largo de los días
  en los diferentes meses?

  ● ¿Cómo varían las ventas a lo largo de los diferentes días de la semana?

  ● ¿Se observa alguna diferencia significativa en las ventas entre los días laborables y
  los fines de semana?

### 6. Impacto de eventos especiales en las ventas:

  ● ¿Se ha observado algún aumento o disminución significativa en las ventas en días
  cercanos a eventos especiales, como días festivos?

  ● ¿Qué eventos específicos han tenido un impacto notable en el comportamiento de
  las ventas y cómo se manifestó ese impacto?